# Create meeting minutes from an Audio file by CPU ONLY

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be DOWNLOADED [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!

## Please note:

When you run the pip installs in the first cell below, you might get this error - it can be safely ignored - it sounds quite severe, but it doesn't seem to affect anything else in this project!


> ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.



In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
# from google.colab import drive
from huggingface_hub import login
# from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
# LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

In [4]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

# drive.mount("/content/drive")
# audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"
audio_filename = "denver_extract.mp3"

# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [5]:
import os
from dotenv import load_dotenv

In [6]:
# Sign in to HuggingFace Hub

# hf_token = userdata.get('HF_TOKEN')
# hf_token = userdata.get('HF_TOKEN')
# load_dotenv(override=True)
load_dotenv(dotenv_path=os.path.join("..", ".env"), override=True)
hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [7]:
# Sign in to OpenAI using Secrets in Colab

# openai_api_key = userdata.get('OPENAI_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [8]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United Sta

In [9]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [10]:
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_quant_type="nf4"
# )

In [11]:
# tokenizer = AutoTokenizer.from_pretrained(LLAMA)
# tokenizer.pad_token = tokenizer.eos_token
# inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cpu")
# streamer = TextStreamer(tokenizer)
# model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto")
# outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token  # still OK

# Apply chat template and tokenize properly
prompt_text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False  # Get raw text, not tensor yet
)

# Tokenize with padding and truncation, and return attention_mask
inputs = tokenizer(
    prompt_text,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Set model manually to CPU if not using GPU
model = AutoModelForCausalLM.from_pretrained(LLAMA).to("cpu")
streamer = TextStreamer(tokenizer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Generate with proper attention mask
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=2000,
    streamer=streamer,
    pad_token_id=tokenizer.pad_token_id  # explicitly set pad_token_id
)

response = tokenizer.decode(outputs[0])

display(Markdown(response))
del outputs

# Use Gradio UI to run the Meeting Minutes - Exercise...

In [17]:
!pip install -q gradio
import gradio as gr

In [18]:
# Transcribe the uploaded audio file using OpenAI's Whisper model
def transcribe_audio(audio_path, progress=gr.Progress()):
    progress(0.3, desc="Creating transcript from audio...")

    try:
        with open(audio_path, "rb") as audio_file:
            transcription = openai.audio.transcriptions.create(
                model=AUDIO_MODEL,
                file=audio_file,
                response_format="text"
            )
            return transcription
    except Exception as e:
        return f"Error during transcription: {str(e)}"

In [19]:
# Generate meeting minutes
def generate_minutes(transcription, model, tokenizer, progress=gr.Progress()):
    progress(0.6, desc="Generating meeting minutes from transcript...")

    system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
    user_prompt = f"Below is an extract transcript of a meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Apply chat template and tokenize properly
    prompt_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False  # Get raw text, not tensor yet
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Tokenize with padding and truncation, and return attention_mask
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=2000,
        streamer=streamer,
        pad_token_id=tokenizer.pad_token_id  # explicitly set pad_token_id
    )
    
    response = tokenizer.decode(outputs[0])

    # Clean up the response, keep only the minutes
    progress(0.9, desc="Cleaning and formatting minutes...")
    response = response.split("<|end_header_id|>")[-1].strip().replace("<|eot_id|>","")

    return response

In [20]:
# Process the uploaded audio file, transcribe it, and generate meeting minutes
def process_upload_transcribe_generate(audio_file, progress=gr.Progress()):
    progress(0.1, desc="Starting process...")

    if audio_file is None:
        return "Please upload an audio file."

    try:
        # Check file format
        if not str(audio_file).lower().endswith('.mp3'):
            return "Please upload an MP3 file."

        # Get transcription via transcribe_audio
        transcription = transcribe_audio(audio_file)
        if transcription.startswith("Error"):
            return transcription

        # Generate minutes via generate_minutes
        minutes = generate_minutes(transcription, model, tokenizer)
        progress(1.0, desc="Process complete!")
        return minutes

    except Exception as e:
        return f"Error processing file: {str(e)}"

### Using Gradio ...

In [21]:
# Create Gradio interface
interface = gr.Interface(
    fn=process_upload_transcribe_generate,
    inputs=gr.Audio(type="filepath", label="Upload MP3 File", format="mp3"),
    outputs=gr.Markdown(label="Meeting Minutes", min_height=60),
    title="Meeting Minutes Generator",
    description="Upload an MP3 recording of your meeting to get AI-generated meeting minutes. This process may take a few minutes.",
    flagging_mode="never"
)

In [22]:
# Launch Gradio interface

interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings

# Student contribution

Student Emad S. has made this powerful variation that uses `TextIteratorStreamer` to stream back results into a Gradio UI, and takes advantage of background threads for performance! I'm sharing it here if you'd like to take a look at some very interesting work. Thank you, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D

## Alternative implementation

Class student Youssef has contributed this variation in which we use an open-source model to transcribe the meeting Audio.

Thank you Youssef!

In [24]:
del model, tokenizer

NameError: name 'model' is not defined

In [25]:
!pip install -q transformers datasets tokenizers

In [26]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [27]:
# AUDIO_MODEL = "openai/whisper-medium"
AUDIO_MODEL = "openai/whisper-tiny.en"
# speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL,
    torch_dtype=torch.float32,  # for CPU
    use_safetensors=True
)

speech_model.to('cpu')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=-1  # ✅ Use -1 for CPU
)

Device set to use cpu


In [28]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename, return_timestamps=True)

D:\development\llm_engineering\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [29]:
transcription = result["text"]
print(transcription)

 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason the back of the logo is considered water, so let you see the creation of the logo here. And yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind and to bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th, please rise with the Pledge of Allegiance by Councilman Lopez. Congratulations to the flag of the Unite

In [30]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cpu")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto")
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason the back of the logo is considered water, so let you see the creation of the logo here. And yeah, so that basically kind of sums up the reason behind the logo and all the meani

In [31]:
response = tokenizer.decode(outputs[0])
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 May 2025

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason the back of the logo is considered water, so let you see the creation of the logo here. And yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind and to bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th, please rise with the Pledge of Allegiance by Councilman Lopez. Congratulations to the flag of the United States of America. Is there a republic which stands one main street under God in the visible liberty and justice for all. All right, thank you, Councillor Mallowpez, Madam Secretary, we'll roll call. Black. Clerk. Here. Espinosa. Here. Flynn. Gilmore. Here. Here. Cashman. Here. Can each. Here. Lopez. New? Yeah. Ortega? Sir? Mr. President. Here. 11. 11. 11. President, we do have a quorum. Approve of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none, minutes of October 2nd. Then approve council announcements. Are there any announcements by members of council? and Clark. Thank you, Mr. President. I just want to do invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock PM. It will move along Broadway from 3rd to Alameda. It's going to be a fun family friendly event. Everyone's invited to come down. Where costume. There will be candy for the kids and there are teacys ambies and 29 and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween. Thank you, Mr. President. All right, thank you, Councilman Clark. I will be there. All right, presentations. Mount Secretary, do we have any presentations? None, Mr. President. Communications, do we have any communications? None, Mr. President. We do have one proclamation this evening, proclamation 1127, an observance of the annual Indigenous Peoples Day in the city and county of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President. With pride. Proclamation number 17, well, let me to say this differently. Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the city and county of Denver. Whereas the council of the city and county of Denver recognizes that the indigenous peoples have lived and flourished on the lands known as the Amedicus and its time and memorial. And that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes, which include the southern youth, the youth mountain, youth tribes of Colorado, and whereas the tribal homelands and seasonal encampments of the Arapaho and Shamp people along the banks So the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3rd, 2016, the city and county of Denver, unanimously passed Council Bill 801, series of 2016 officially desulating the second Monday of October of each year as indigenous peoples day in Denver, Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through indigenous peoples knowledge, science, philosophy, arts, and culture, and through these contributions, the city of Denver has developed and thrived. Whereas the indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of indigenous people including Confluence Week, drawing record attendance to a national indigenous youth leadership conference leading conversations on inclusion with their peers and supporting increased Indigenous youth participation in science and engineering. Now therefore, be it proclaimed by the Council of the City and County of Denver, section one, at the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of indigenous people to our history, our past, our present, and future. And continues to promote the education of the Denver community about these historical and contemporary contributions of Indigenous people. Section two, at the City and County of Denver, Colorado does hereby observe October 9, 2017, as Indigenous Peoples Day. Section three, at the clerk of the City and County of Denver, shall attest and affix the seal of the City and County of Denver to this proclamation and that a copy be transmitted, transmitted, excuse me, to the Denver American Indian Commission, the City and County of Denver School District number one and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez, your motion to adopt. Mr. President, I move that proclamation number 1127, series of 2017, be adopted. All right, it has been moved and seconded. Councilor, members of Councilor Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for this, for the third time, but as Indigenous Peoples Day and Denver, officially for the second time. It is always awesome to be able to see, not just this proclamation come by my desk, but to see the so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier, and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day when we sat down and wrote it and as a community we couldn't think of anything else to begin except for the confluence of the two rivers and those confluence of the two rivers created such a great city and we live in such an amazing city and we we're all proud of it and sometimes we and a lot of people from all over the country are out over the world are proud of it and sometimes a little too proud of it is it's time to go back home but I'm well I'm kidding when I say that but the really nice thing about this is that we are celebrating indigenous people's day out of pride for who we are who we are as a city and the contributions of indigenous people to the city not out of spite not out of a replacement of one culture over the other or out of contempt or disrespect. I think of a quote that Sesso Chavez made very popular and it stuck with me for a very long time and any time I have the opportunity I speak in front of children and especially the children in our community that they often second guess themselves in where they're coming from, who they are. And I always say that it's very important to be proud of who we are from. And the quote that I use from Sister Chavez is, pride in one's own culture does not require contempt or the disrespect of another. And that's very important. It's very important for us to recognize that, No matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year to be for that to sit on our shoulders for a while, for us to think about, right? And so I wanted to just to thank you all, to thank the commission. There's going to be a couple individuals that are going to come speak. Thank you for your art, your lovely artwork, for us to see. what's in your heart and what now has become, probably is gonna be a very important symbol for the community. And also just for the work, the daily work, every single day, we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty, without access to services, right, without access to sobriety or even housing or jobs, And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations in this day is not a day off, it's a day on and done. And addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation. I know you always have done in the past. I'm very proud of today. Oh, and you made time magazine and Newsweek once again today. As being a leader in terms of the cities that are celebrating and business people's day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martiga? Mr. President, I want to ask if my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the youths and the commensities that had been sort of at odds with each other for about 100 years and just being able to participate in that powwow was pretty awesome. So for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the native language from generation to generation is just so incredible because in so many cultures, you know, people have come here and assimilated to the, you know, the norms here and they lose their language and lose a lot of the culture. in the native community, that hasn't happened. That commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right, thank you, Councilwoman Arttega. Councilwoman Kenee. Thank you very much. And I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was in your description of it. And I think one of the things that is clear is, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture. And so I think the art is so important. And when you talked about water, I was also thinking about land. And I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront or actually nationally of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect, but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation in all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here. And I think that I just want to mention a solidarity. And I mentioned a feeling of solidarity with that resistance. So thank you. And happy Confluence Week. Thank you, Councilwoman Keneech. And see you in other comments. I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous people's days, because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil types, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
=============================================

**Date:** Monday, October 9th
**Location:** Denver City Council Chambers
**Attendees:**

* Council Members: Black, Espinosa, Flynn, Gilmore, Cashman, Lopez, Martiga, Kenee
* Clerk: Espinosa
* President: President (unspecified)
* Guests: Various individuals and representatives from Native American communities

**Summary:**
The Denver City Council met to discuss and adopt a proclamation recognizing Indigenous Peoples Day. Councilman Lopez presented a proclamation (1127, series of 2017) and highlighted the importance of acknowledging the contributions of indigenous peoples to the city's history and culture. The proclamation was adopted by the Council, and Council members expressed their support and appreciation for the Native American community.

**Discussion Points:**

* The importance of recognizing indigenous peoples and their contributions to the city's history and culture
* The creation of the logo for Confluence Week, which represents the merging of two rivers and the city's founding
* The need for inclusivity and respect for all cultures and communities
* The significance of Native American communities in defending public lands and promoting environmental protection

**Takeaways:**

* The city recognizes the importance of acknowledging and celebrating indigenous peoples and their contributions
* The proclamation adopted by the Council will promote education and awareness about indigenous peoples and their history
* The city will continue to support and recognize the contributions of Native American communities

**Action Items:**

* Councilman Lopez: Sponsor and present the proclamation (1127, series of 2017) for adoption
* Council Members: Express their support and appreciation for the Native American community and the proclamation
* Clerk: Ensure that a copy of the proclamation is transmitted to the Denver American Indian Commission, the City and County of Denver School District number one, and the Colorado Commission on Indian Affairs
* Various individuals and representatives from Native American communities: Continue to share their culture, traditions, and contributions with the community

**Next Steps:**
The proclamation (1127, series of 2017) will be implemented and promoted by the city. The city will continue to support and recognize the contributions of Native American communities, and the Confluence Week logo will be used to promote awareness and education about indigenous peoples and their history.<|eot_id|>